In [6]:
import os
from langchain_community.utilities.bing_search import BingSearchAPIWrapper
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import tool
from langchain_openai import AzureChatOpenAI
from langchain_community.tools.bing_search import BingSearchRun
# from langchain_experimental.tools import PythonREPLTool
from langchain_core.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent
from langchain.vectorstores.qdrant import Qdrant
from langchain.tools.retriever import create_retriever_tool
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.agents import AgentExecutor
from langchain_experimental.utilities import PythonREPL
from langchain.agents import Tool
# replace with your keys
os.environ["BING_SUBSCRIPTION_KEY"] = "Your key"
os.environ["BING_SEARCH_URL"] = "https://api.bing.microsoft.com/v7.0/search"

In [3]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
# Load embedding model for retriever

# This chosen model is specialized for Chinese, replace it if you are using other languages
model_name = "amu/tao-8k"
model_kwargs = {"device": "cuda"} # change to cpu if your device has no cuda support
encode_kwargs = {"normalize_embeddings": True}
hf_embedding = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
) # If you do not want to download the model everytime, add the parameter 'cache_folder' to cache the model

No sentence-transformers model found with name C:\Users\a1383/.cache\torch\sentence_transformers\amu_tao-8k. Creating a new one with MEAN pooling.


In [4]:
# Define retreiver tool
loader = CSVLoader(file_path='adcode_citycode.csv', encoding="UTF-8")
data = loader.load()
qdrant = Qdrant.from_documents(
    data,
    hf_embedding,
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="citycodes",
)
retriever = qdrant.as_retriever()
retriever_tool = create_retriever_tool(
    retriever,
    "city_to_code_search",
    "城市名对应的城市编码，天气查询时必须要先使用该tool获取城市名对应的编码",
)

In [8]:
# Define Gaode weather search tool
class WeatherSearchReactInput(BaseModel):
    inputs: str = Field(description="执行搜素所需要的输入")

@tool("weather_react_search-tool", args_schema=WeatherSearchReactInput)
async def weather_react_search(input:str) -> str:
    """
    天气查询
    使用该tool时必须确保已经使用city_to_code_search tool获取到了对应城市编码
    input: str 执行搜素所需要的输入, input有三个key, {'city': str 查询的城市编码, 'extensions': str 气象类型 'base':返回实况天气, 'all':返回预报天气, 'output': str 可选值：JSON,XML}.
    其中，city为必须存在的字段。extensions的默认值为'base', output的默认值为'JSON'
    """

    import requests
    import ast
    input = ast.literal_eval(input)
    url = "https://restapi.amap.com/v3/weather/weatherInfo?"
    params = {
        "key": "Your key", # Replace with your key
        "city": input.get("city"),
        "extensions": input.get("extensions"),
        "output": input.get("output")
    }
    resp = requests.get(url, params=params).json()
    if resp.get("status") != "1":
        raise Exception("高德天气接口请求失败")
    if resp.get("forecasts"):
        
        return str(resp.get("forecasts"))
    elif resp.get("lives"):
        
        return str(resp.get("lives")[0])

In [9]:
# Define bing search wrapper, change the mkt parameter based on your search region
search = BingSearchAPIWrapper(search_kwargs={"mkt": "zh-CN"})

In [10]:
# Define python repl tool for code writing
python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="一个Python shell。使用它来执行Python命令。输入应该是一个有效的Python命令。如果你想看到一个值的输出，你应该使用print(...)来打印它。",
    func=python_repl.run,
)

In [17]:
async def react_agent_run(query):
    """
    The running function for the react agent. Change the chatLLM model as you wish.

    query: str - The user input

    """

    # Define the chat model, I am using azure here and all credential are set in my environment, so you need to config yours
    azure_api_type="azure"
    azure_api_version="2023-08-01-preview"
    temperature=0
    model = AzureChatOpenAI(
                    openai_api_type=azure_api_type,
                    openai_api_version=azure_api_version,
                    model_name="Your deployment name",
                    temperature=temperature,
                )
    # Define the toolkit
    tools = [weather_react_search, retriever_tool, BingSearchRun(api_wrapper=search, description="用来搜索实时信息，一定要以中文进行搜索"), repl_tool]

    instructions = """
    你是一个会合理利用不同工具来帮助用户解决问题的智能助手，来自九号AI科技公司。你必须要仔细思考整个解答过程，拆分用户不同需求，包括是否需要调用工具，调用哪些工具，如何利用工具的输出结果等。

    任何关于代码的问题或者涉及到计算的问题，你都需要使用提供的python REPL工具，来编写和运行代码来帮助你回答问题。

    关于使用python REPL工具的注意事项：

    - 如果你遇到错误，请调试你的代码并再试一次。

    - 只使用你代码的输出来回答问题。

    - 你可能在不运行任何代码的情况下知道答案，但你仍应运行代码以验证你的答案。

    如果没有合适的工具来帮助你获得需要的信息，则告诉用户你当前没有这个能力回答该问题。
    
    可用工具：
    ------
    你可以使用以下工具：
    {tools}

    执行Action时，应为[{tool_names}]中的其中一个。注意！这里必须只放工具名称，不要有其他内容

    使用工具时，你必须需要使用以下格式：
    ```
    Thought: 我是否需要使用工具? Yes
    Action: 你所选的工具名称要放在这里
    Action Input: 执行该行动所需要的输入
    Observation: 执行该行动所获得的结果或答案
    ```

    当你准备好向用户输出答案，或者不需要使用任何工具时，你必须使用以下格式：
    ```
    Thought: 我是否需要使用工具? No
    Final Answer: [你的最终答案要放在这里]
    ```

    Begin!

    以前的聊天历史：
    {chat_history}

    ------
    New input: {input}
    {agent_scratchpad}
    """
    
    prompt = PromptTemplate(
        input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'],
        template=instructions,
        partial_variables={"chat_history": ""}
    )
    prompt.partial(chat_history="")
    
    agent = create_react_agent(model, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True).with_config({"run_name": "Agent"})
    res = await agent_executor.ainvoke({"input": query})
    return res
    # The following code enabled the stream, enable if you need
    #-----------------------------------------------------------
    # async for event in agent_executor.astream_events(
    #     {"input": query},
    # ):
    #     kind = event["event"]
    #     if kind == "on_chain_start":
    #         if (
    #             event["name"] == "Agent"
    #         ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
    #             print(
    #                 f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
    #             )
    #     elif kind == "on_chain_end":
    #         if (
    #             event["name"] == "Agent"
    #         ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
    #             print()
    #             print("--")
    #             print(
    #                 f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
    #             )
    #     if kind == "on_chat_model_stream":
    #         content = event["data"]["chunk"].content
    #         if content:
    #             # Empty content in the context of OpenAI means
    #             # that the model is asking for a tool to be invoked.
    #             # So we only print non-empty content
    #             print(content, end="|")
    #     elif kind == "on_tool_start":
    #         print("--")
    #         print(
    #             f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
    #         )
    #     elif kind == "on_tool_end":
    #         print(f"Done tool: {event['name']}")
    #         print(f"Tool output was: {event['data'].get('output')}")
    #         print("--")
    

In [18]:
res = await react_agent_run("南阳今天的天气怎么样")



> Entering new AgentExecutor chain...
```
Thought: 我是否需要使用工具? Yes
Action: city_to_code_search
Action Input: 南阳[Document(page_content='\ufeff中文名: 南阳市\nadcode: 411300\ncitycode: 0377', metadata={'source': 'adcode_citycode.csv', 'row': 1605}), Document(page_content='\ufeff中文名: 汝南县\nadcode: 411727\ncitycode: 0396', metadata={'source': 'adcode_citycode.csv', 'row': 1659}), Document(page_content='\ufeff中文名: 南召县\nadcode: 411321\ncitycode: 0377', metadata={'source': 'adcode_citycode.csv', 'row': 1608}), Document(page_content='\ufeff中文名: 南陵县\nadcode: 340223\ncitycode: 0553', metadata={'source': 'adcode_citycode.csv', 'row': 1025})]Action: weather_react_search-tool
Action Input: {'city': '411300', 'extensions': 'base', 'output': 'JSON'}{'province': '河南', 'city': '南阳市', 'adcode': '411300', 'weather': '霾', 'temperature': '3', 'winddirection': '东', 'windpower': '≤3', 'humidity': '64', 'reporttime': '2024-02-07 11:30:11', 'temperature_float': '3.0', 'humidity_float': '64.0'}Final Answer: 南阳今天的天气是霾

In [19]:
print(res)

{'input': '南阳今天的天气怎么样', 'output': '南阳今天的天气是霾，气温3摄氏度，湿度64%。'}
